In [1]:
### Let's build a convolutional neural network for the 3-label classification ###

In [2]:
# import modules
import pandas as pd
import numpy as np
import cv2 as cv
from IPython.display import clear_output
import matplotlib.pyplot as plt 
import os

In [3]:
# Load training data csv. The original file contains the images names and patient data
training_data_csv = pd.read_csv('../merged_training_data.csv')
training_data_csv.head()

# Load an image and mask
img = cv.imread('../../ISIC-2017_Training_Data/ISIC_0000100.jpg',-1) 
mask = cv.imread('../../ISIC-2017_Training_Part1_GroundTruth/ISIC_0000100_segmentation.png',-1)

# Color conversion and normalization
output_array = np.zeros((img.shape[0],img.shape[1])) # Array to store normalized image
img = cv.cvtColor(img, cv.COLOR_BGR2RGB) 

## Segmentation
segmented = cv.normalize(cv.bitwise_and(img, img, mask=mask),  output_array, 0, 255, cv.NORM_MINMAX)
plt.imshow(segmented);

In [4]:
## 0 - neither, 1 - melanoma, 2 - seborrheic keratosis
for i,u in enumerate(training_data_csv['image_id']):
    if training_data_csv.loc[i,'melanoma'] == 0:
        if training_data_csv.loc[i,'seborrheic_keratosis'] == 1:
            training_data_csv.loc[i,'melanoma'] = 2
    else:
        continue
training_data_csv.drop(columns=['seborrheic_keratosis'], inplace=True)
training_data_csv.head()

In [5]:
half = cv.resize(segmented, (200, 200))
plt.imshow(half)

In [6]:
### Segment, normalize and resize each image
### It will also separate images from different classes in different folders

im_path = '../../ISIC-2017_Training_Data/' ## CHANGE THIS VARIABLE WITH THE PATH WHERE YOU HAVE THE IMAGES
mask_path = '../../ISIC-2017_Training_Part1_GroundTruth/' ## CHANGE THIS VARIABLE WITH THE PATH WHERE THE MAKS ARE

melanoma_dir = 'melanoma/'
seborrheic_keratosis_dir = 'seborrheic keratosis/'
neither_dir = 'neither/'

try:
    os.mkdir('../../segmented/')

    os.mkdir('../../segmented/' + melanoma_dir)
    os.mkdir('../../segmented/' + seborrheic_keratosis_dir)
    os.mkdir('../../segmented/' + neither_dir)
    
except:
    pass

try:
    os.mkdir('../../segmented_full/')
except:
    pass
    

## function to segment, normalize, resize and save images
for index, image in enumerate(training_data_csv['image_id']):
    
    # read
    img = cv.imread(im_path + image + '.jpg',-1) 
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
    # segment & resize
    mask = cv.imread(mask_path + image + '_segmentation.png', -1)    
    output_array = np.zeros((img.shape[0],img.shape[1])) # Array to store normalized image
    
    ### resize the segmented image to 150x150
    img_segmented = cv.resize(cv.normalize(cv.bitwise_and(img_rgb, img_rgb, mask=mask),  output_array, 0, 255, cv.NORM_MINMAX),
                              (200, 200))
    
    # save the images in a folder
    if training_data_csv.loc[index,'melanoma'] == 0:
        cv.imwrite('../../segmented/' + neither_dir + 'SEG' + image + '.jpg' , cv.cvtColor(img_segmented, cv.COLOR_RGB2BGR))
    elif training_data_csv.loc[index,'melanoma'] == 1:
        cv.imwrite('../../segmented/' + melanoma_dir + 'SEG' + image + '.jpg' , cv.cvtColor(img_segmented, cv.COLOR_RGB2BGR))
    else:
        cv.imwrite('../../segmented/' + seborrheic_keratosis_dir + 'SEG' + image + '.jpg' , cv.cvtColor(img_segmented, cv.COLOR_RGB2BGR))
    
    cv.imwrite('../../segmented_full/SEG' + image + '.jpg' , cv.cvtColor(img_segmented, cv.COLOR_RGB2BGR))
    clear_output(wait=True)
    print('Segmented image ' + str(index+1) + '/'+ str(len(training_data_csv['image_id'])) + ' saved.')
    
print('All images have been segmented and saved')